In [49]:
import pandas as pd
import numpy as np
import os

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Saving figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Load the data

In [50]:
ECOMMERCE_DATA = "datasets"

In [84]:

def load_ecommerce_data(ecommerce_path=ECOMMERCE_DATA):
    csv_path = os.path.join(ecommerce_path, "ecommerce_data.csv")
    return pd.read_csv(csv_path, encoding="ISO-8859-1", dtype={'CustomerID': str})


## Checking the structure of the data

In [85]:
ecommerce = load_ecommerce_data()

In [86]:
ecommerce.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


8 columns for the data. Each row represents an item of an order:
1. InvoiceNo   : the invoice number of the order
2. StockCode   : stock number for the product
3. Description : product's description
4. Quantity    : product's quantity in the order
5. InvoiceDate : order date or the issued date of the invoice
6. UnitPrice   : price per unit
7. CustomerID  : customer's id
8. Country     : customer's country (strongly)

In [87]:
ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


Quick facts:
* 541909 entries
* Numerical values: Quantity, UnitPrice, CustomerID
* InvoiceNo: string
* Description missing ~1.4k values
* CustomerID missing at leasst 13k values


Negative values? Cancelled orders or updated orders 

Removing the orders with no customer id (or just keeping those that are not na)

In [88]:
ecommerce = ecommerce[ecommerce['CustomerID'].notna()]

In [89]:
# remove the dupliucates
ecommerce = ecommerce.drop_duplicates()

In [90]:
ecommerce.shape

(401604, 8)

We left with 401604 rows 

In [92]:
ecommerce.InvoiceDate = ecommerce.InvoiceDate.astype('datetime64')

In [93]:
ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  object        
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.6+ MB


Removing the empty Customer IDs also removed empty descriptions


In [94]:
#make the Customer IDs into int and then to Strings


In [95]:
ecommerce.describe()

,Quantity,UnitPrice
count,401604.000000,401604.000000
mean,12.183273,3.474064
std,250.283037,69.764035
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


In [96]:
ecommerce.CustomerID.value_counts()

17841    7812
14911    5898
14096    5128
12748    4459
14606    2759
         ... 
18068       1
13256       1
15590       1
16138       1
15389       1
Name: CustomerID, Length: 4372, dtype: int64

quick glance on the customers and their number of orders

In [97]:
ecommerce.Description.value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2058
REGENCY CAKESTAND 3 TIER              1894
JUMBO BAG RED RETROSPOT               1659
PARTY BUNTING                         1409
ASSORTED COLOUR BIRD ORNAMENT         1405
                                      ... 
PINK/WHITE GLASS DEMI CHOKER             1
 I LOVE LONDON MINI RUCKSACK             1
SET 36 COLOURING PENCILS DOILEY          1
SET 12 COLOURING PENCILS DOILEY          1
PEARL AND CHERRY QUARTZ BRACLET          1
Name: Description, Length: 3896, dtype: int64

quick glance on the items and how often are found in an order

In [98]:
ecommerce.Description.describe()

count                                 401604
unique                                  3896
top       WHITE HANGING HEART T-LIGHT HOLDER
freq                                    2058
Name: Description, dtype: object

In [110]:
ecommerce[ecommerce.Quantity<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


not only cancelations. Also Discounts
For every cancelation the invoice starts with a 'C'

In [113]:
ecommerce[ecommerce.Quantity<0].Description.value_counts()

REGENCY CAKESTAND 3 TIER               180
Manual                                 175
POSTAGE                                 97
JAM MAKING SET WITH JARS                86
Discount                                77
                                      ... 
FOLDING UMBRELLA CREAM POLKADOT          1
JUMBO BAG CHARLIE AND LOLA TOYS          1
LARGE CAKE TOWEL PINK SPOTS              1
AIRLINE BAG VINTAGE WORLD CHAMPION       1
PAPER CRAFT , LITTLE BIRDIE              1
Name: Description, Length: 1945, dtype: int64

In [116]:
ecommerce[ecommerce.Description == 'Manual']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274,United Kingdom
6798,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723,United Kingdom
7976,537077,M,Manual,12,2010-12-05 11:59:00,0.42,17062,United Kingdom
8530,537137,M,Manual,36,2010-12-05 12:43:00,0.85,16327,United Kingdom
...,...,...,...,...,...,...,...,...
531432,580956,M,Manual,4,2011-12-06 14:23:00,1.25,17841,United Kingdom
533082,C581009,M,Manual,-1,2011-12-07 09:15:00,125.00,16971,United Kingdom
534308,C581145,M,Manual,-1,2011-12-07 13:48:00,9.95,17490,United Kingdom
538321,581405,M,Manual,3,2011-12-08 13:50:00,0.42,13521,United Kingdom


In [135]:
df = ecommerce[ecommerce.CustomerID == '16274']

df.head(50)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2185,536569,22171,3 HOOK PHOTO SHELF ANTIQUE WHITE,1,2010-12-01 15:35:00,8.50,16274,United Kingdom
2186,536569,21462,"NURSERY A,B,C PAINTED LETTERS",1,2010-12-01 15:35:00,6.75,16274,United Kingdom
2187,536569,22622,BOX OF VINTAGE ALPHABET BLOCKS,1,2010-12-01 15:35:00,9.95,16274,United Kingdom
2188,536569,22616,PACK OF 12 LONDON TISSUES,12,2010-12-01 15:35:00,0.29,16274,United Kingdom
2189,536569,22713,CARD I LOVE LONDON,12,2010-12-01 15:35:00,0.42,16274,United Kingdom
2190,536569,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 15:35:00,4.95,16274,United Kingdom
2191,536569,22496,SET OF 2 ROUND TINS DUTCH CHEESE,1,2010-12-01 15:35:00,2.95,16274,United Kingdom
2192,536569,84870C,GREEN GEISHA GIRL,1,2010-12-01 15:35:00,3.75,16274,United Kingdom
2193,536569,22659,LUNCH BOX I LOVE LONDON,1,2010-12-01 15:35:00,1.95,16274,United Kingdom
2194,536569,22494,EMERGENCY FIRST AID TIN,1,2010-12-01 15:35:00,1.25,16274,United Kingdom


In [137]:
df.tail(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2237,536569,21871,SAVE THE PLANET MUG,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
2238,536569,21071,VINTAGE BILLBOARD DRINK ME MUG,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
2240,536569,21340,CLASSIC METAL BIRDCAGE PLANT HOLDER,1,2010-12-01 15:35:00,12.75,16274,United Kingdom
2241,536569,22816,CARD MOTORBIKE SANTA,12,2010-12-01 15:35:00,0.42,16274,United Kingdom
2242,536569,22818,CARD CHRISTMAS VILLAGE,12,2010-12-01 15:35:00,0.42,16274,United Kingdom
2243,536569,21591,COSY HOUR CIGAR BOX MATCHES,2,2010-12-01 15:35:00,1.25,16274,United Kingdom
2244,536569,21587,COSY HOUR GIANT TUBE MATCHES,3,2010-12-01 15:35:00,2.55,16274,United Kingdom
2245,536569,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-01 15:35:00,3.75,16274,United Kingdom
2246,536569,22767,TRIPLE PHOTO FRAME CORNICE,1,2010-12-01 15:35:00,9.95,16274,United Kingdom


Only one invoice for 16274 customer id and no cancelation. Still what is the Manual??


TODO: In order to proceed I will remove the orders that are below 0; orders that are cancelled or discounted,

NOTE! Normally I should have gone with:
* Removing the orders that are definitelly cancelled, such as those that are not Discounted and the negative quantity is the same value as a positive one for the same customer in  an older date/invoice

In [152]:
# let's try it anyway
df_to_remove = ecommerce[(ecommerce.Quantity<0) & (ecommerce.StockCode != 'D') & (ecommerce.StockCode != 'M')]

In [153]:
df_to_remove

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
540448,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [164]:
num = 0
for index, row in df_to_remove.iterrows():

    df_temp = (ecommerce[(ecommerce.StockCode == row.StockCode) & (ecommerce.CustomerID == row.CustomerID) & (ecommerce.InvoiceDate < row.InvoiceDate)])
    
    if df_temp.empty:
        print('empty')
    else:
        print('notempty')
        break

empty
empty
empty
empty
empty
empty
empty
empty
notempty
